In [1]:
import xml.etree.ElementTree as ET
from xml.dom import minidom

In [2]:
def prettify(elem):
    rough_string = ET.tostring(elem, 'utf-8')
    reparsed = minidom.parseString(rough_string)
    return reparsed.toprettyxml(indent="  ")

In [3]:
# input example
data = [[('Friday', '02:00:00-10:00:00'),
          ('Monday', '02:00:00-10:00:00'),
          ('Thursday', '02:00:00-10:00:00'),
          ('Tuesday', '02:00:00-10:00:00'),
          ('Wednesday', '02:00:00-10:00:00'),
          'Sachbearbeiter_#0'],
         [('Friday', '02:00:00-10:00:00'),
          ('Monday', '02:00:00-10:00:00'),
          ('Thursday', '02:00:00-10:00:00'),
          ('Tuesday', '02:00:00-10:00:00'),
          ('Wednesday', '02:00:00-10:00:00'),
          'Manager_#0']]

In [4]:
# Role and schedules extraction
roles = []
schedules = []
for log in data:
    roles.append(log[-1])
    schedules.append(log[:-1])

In [5]:
# namespace, will be update to tec (ITESM) namesapce
ns = {"bsim": "http://bsim.hpi.uni-potsdam.de/scylla/simModel"}

ET.register_namespace('bsim', ns["bsim"])
root = ET.Element(ET.QName(ns["bsim"], "globalConfiguration"))

resourceData =  ET.Element(ET.QName(ns["bsim"], "resourceData"))
timeTables = ET.Element(ET.QName(ns['bsim'], "timeTables"))
root.append(resourceData)
root.append(timeTables)

# For each role
for index,role in enumerate(roles):
    dynamicResource =  ET.Element(ET.QName(ns["bsim"], "dynamicResource"))
    dynamicResource.set("id", role)
    dynamicResource.set("defaultTimeUnit", "HOURS")
    tt_id = role + '_schedule'
    dynamicResource.set("defaultTimetableId", tt_id)
    # default quantity will be set to 5 until we get an algorithm to retrive the amount of concurrency on resources
    dynamicResource.set("defaultQuantity", "5")
    resourceData.append(dynamicResource)
    current_schedule = schedules[index]
    timeTable = ET.Element(ET.QName(ns['bsim'], "timeTable"))
    timeTables.append(timeTable)
    timeTable.set('id', tt_id)
    for day in current_schedule:
        timeTableItem = ET.Element(ET.QName(ns['bsim'], "timeTableItem"))
        timeTableItem.set('from',day[0].upper())
        timeTableItem.set('to',day[0].upper())
        hours = day[1].split('-')
        timeTableItem.set('beginTime', hours[0][:6])
        timeTableItem.set('endTime', hours[1][:6])
        timeTable.append(timeTableItem)

In [6]:
file = open('scylla_config.xml','w')
file.write(prettify(root))
file.close()